# Web scraping - transfers/contracts

In [2]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

In [3]:
load_dotenv()

def get_soup(url):
    page = requests.get(url,headers = {"User-Agent":"Mozilla/5.0","origin":"https://www.transfermarkt.co.uk"})
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [10]:
team_urls = []
for year in ['2018','2019','2020']:
    transfer_soup = get_soup(f'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={year}')
    main_div = transfer_soup.find('div',{'id':'main'})
    next_div = main_div.find('div',{'id':'wettbewerbsstartseite'})
    team_table = next_div.find('table',{'class':'items'}).find('tbody')
    for team in team_table.find_all('tr'):
        team_link = team.find('td',{'class':'hauptlink'}).find('a')
        link = span.find('a', {'class':'spielprofil_tooltip'})
        team_urls.append(team_link["href"])

team_urls

['/manchester-city/startseite/verein/281/saison_id/2018',
 '/fc-chelsea/startseite/verein/631/saison_id/2018',
 '/fc-liverpool/startseite/verein/31/saison_id/2018',
 '/manchester-united/startseite/verein/985/saison_id/2018',
 '/tottenham-hotspur/startseite/verein/148/saison_id/2018',
 '/fc-arsenal/startseite/verein/11/saison_id/2018',
 '/fc-everton/startseite/verein/29/saison_id/2018',
 '/leicester-city/startseite/verein/1003/saison_id/2018',
 '/west-ham-united/startseite/verein/379/saison_id/2018',
 '/fc-southampton/startseite/verein/180/saison_id/2018',
 '/crystal-palace/startseite/verein/873/saison_id/2018',
 '/fc-fulham/startseite/verein/931/saison_id/2018',
 '/newcastle-united/startseite/verein/762/saison_id/2018',
 '/fc-burnley/startseite/verein/1132/saison_id/2018',
 '/afc-bournemouth/startseite/verein/989/saison_id/2018',
 '/brighton-amp-hove-albion/startseite/verein/1237/saison_id/2018',
 '/fc-watford/startseite/verein/1010/saison_id/2018',
 '/wolverhampton-wanderers/startseit

In [91]:
player_urls = []
for team_url in team_urls:
    team_soup = get_soup('https://www.transfermarkt.co.uk'+team_url)
    main_div = team_soup.find('div',{'id':'main'})
    div = main_div.find('div',{'itemtype':'http://schema.org/SportsTeam'})
    player_table = div.find('table',{'class':'items'}).find('tbody')
    for player in player_table.find_all('tr',{'class':['odd','even']}):
        a = player.find('td',{'class':'posrela'}).find('td',{'class':'hauptlink'}).find('a')
        player_urls.append([a.text,a["href"]])
        
print(player_urls)
#   table = main_div.find('table',{'class':'items'})
#     for team in main_div.find_all('div',{'class':'box'})[3:]:
#         team_name = team.find('div',{'class':'table-header'}).find('h2')
#         transfers = team.find_all('div',{'class':'responsive-table'})
#         transfer_table = transfers[0].findits('tbody')
#         for transfer_table in transfers[0:1]:
#             transfer_table = transfer_table.find('tbody')
#             for tr in transfer_table.find_all('tr'):
#                 td = tr.find('td')
#                 span = td.find('div').find('span')
#                 link = span.find('a', {'class':'spielprofil_tooltip'})
#                 player_urls.append([link.text,link["href"]])

[['Ederson', '/ederson/profil/spieler/238223'], ['Claudio Bravo', '/claudio-bravo/profil/spieler/40423'], ['Arijanet Muric', '/arijanet-muric/profil/spieler/371021'], ['Daniel Grimshaw', '/daniel-grimshaw/profil/spieler/339253'], ['Aymeric Laporte', '/aymeric-laporte/profil/spieler/176553'], ['John Stones', '/john-stones/profil/spieler/186590'], ['Nicolás Otamendi', '/nicolas-otamendi/profil/spieler/54781'], ['Eliaquim Mangala', '/eliaquim-mangala/profil/spieler/90681'], ['Vincent Kompany', '/vincent-kompany/profil/spieler/9594'], ['Philippe Sandler', '/philippe-sandler/profil/spieler/340460'], ['Pablo Marí', '/pablo-mari/profil/spieler/210178'], ['Erik Palmer-Brown', '/erik-palmer-brown/profil/spieler/282199'], ['Eric García', '/eric-garcia/profil/spieler/466794'], ['Cameron Humphreys', '/cameron-humphreys/profil/spieler/277723'], ['Benjamin Mendy', '/benjamin-mendy/profil/spieler/157495'], ['Oleksandr Zinchenko', '/oleksandr-zinchenko/profil/spieler/203853'], ['Kyle Walker', '/kyle-w

In [105]:
import re
def get_player_transfers(player):
    soup_url = player[1].replace('/profil/','/transfers/')
    player_info = {'date of birth/age':'','place of birth':'',
                   'citizenship':'','height':'','position':'',
                   'agent':'','former international':'',
                   'current international':'','caps/goals':'','contract expires':'','national player':''}
    player_soup = get_soup('https://www.transfermarkt.co.uk' +soup_url)
    main_div = player_soup.find('div',{'id':'main'})
    player_details = main_div.find('div',{'itemtype':'https://schema.org/Person'}).find('div',{'class':'dataContent'}).find('div',{'class':'dataBottom'})
    for p in player_details.find_all('p'):
        item = p.find('span',{'class':'dataItem'}).text
        item = re.sub("\(\d+\)","",item).replace(':',"").strip().lower()
        value = p.find('span',{'class':'dataValue'}).text
        value = re.sub("\(\d+\)","",value).strip()
        player_info[item] = value
    transfer_history = player_soup.find_all('tr', {'class':'zeile-transfer'})
    transfer_list = []
    for tr in transfer_history:
        td = tr.find_all('td')
        player_name = [player[0]]
        row = player_name + [tr.text.strip() for tr in td if tr.text.strip() != '']
        for key in player_info:
            row.append(player_info[key])
        transfer_list.append(row)
    df = pd.DataFrame(transfer_list, columns = ["name","season","date","left","joined","mv","fee","dob","city","country","height","position","agent","former_international","current_international","caps/goals","contract_expires","national_player"])
    return df

In [106]:
import time
import random

df = pd.DataFrame([], columns = ["name","season","date","left","joined","mv","fee","dob","city","country","height","position","agent","former_international","current_international","caps/goals","contract_expires"])
for player in player_urls:
    player_df = get_player_transfers(player)
    df = pd.concat([df,player_df])
    time.sleep(random.randint(3, 10)) # to not break their website

In [108]:
df.drop_duplicates().to_csv('../CSVs/transfers_complete.csv')